In [48]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import zscore
from math import radians, cos, sin, asin, sqrt
import pydot
import seaborn as sns
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Input,Dense, Dropout, Flatten, Activation, Concatenate, Embedding
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model

In [49]:
label_column = 'price'
kc_raw_data = pd.read_csv('kc_house_data.csv')

In [50]:
kc_raw_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### Feature Engineering

In [51]:
kc_raw_data['sale_yr'] = pd.to_numeric(kc_raw_data.date.str.slice(0,4))
kc_raw_data['sale_month'] = pd.to_numeric(kc_raw_data.date.str.slice(4,6))
kc_raw_data['sale_day'] = pd.to_numeric(kc_raw_data.date.str.slice(6,8))
kc_data = pd.DataFrame(kc_raw_data,columns=[
        'sale_yr','sale_month','sale_day', 'view', 'waterfront', 'lat', 'long',
        'bedrooms','bathrooms','sqft_living','sqft_lot','floors',
        'condition','grade','sqft_above','sqft_basement','yr_built','yr_renovated',
        'zipcode','sqft_living15','sqft_lot15','price'])

### Split into train and test

In [5]:
kc_data = kc_data.sample(frac=1)

train = kc_data.sample(frac=0.8)
print('Train:' + str(train.shape))
validate = kc_data.sample(frac=0.1)
print('Validate:' + str(validate.shape))
test = kc_data.sample(frac=0.1)
print('Test:' + str(test.shape))
xsize = kc_data.shape[1]-1
print ('xsize:' + str(xsize))

Train:(17290, 22)
Validate:(2161, 22)
Test:(2161, 22)
xsize:21


### Build a DL model
The model has two hidden dense layers and a final linear combination layer that produces that regression output. We’ll use the mean squared error as the loss function.

In [6]:
t_model = Sequential()
t_model.add(Dense(100, activation="relu",input_shape=(xsize,)))
t_model.add(Dense(50,activation="relu"))
t_model.add(Dense(1,kernel_initializer='normal'))
t_model.compile(
    loss="mean_squared_error",
    optimizer=Adam(lr=0.001),
    metrics=[metrics.mae]
)

In [7]:
t_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               2200      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 7,301
Trainable params: 7,301
Non-trainable params: 0
_________________________________________________________________


### Model training

In [8]:
epochs = 500
batch = 128

cols = list(train.columns)
cols.remove(label_column)
history = t_model.fit(
    train[cols], train[label_column],
    batch_size=batch,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    validation_data=(validate[cols],validate[label_column])
    #callbacks=keras_callbacks
)
score = t_model.evaluate(test[cols], test[label_column], verbose=0)

Train on 17290 samples, validate on 2161 samples
Epoch 1/500
17290/17290 [==============================] - 2s 89us/step - loss: 248210281972.6880 - mean_absolute_error: 347161.7099 - val_loss: 146490766447.3559 - val_mean_absolute_error: 237006.8463
Epoch 2/500
17290/17290 [==============================] - 1s 52us/step - loss: 128671616854.8534 - mean_absolute_error: 226965.6202 - val_loss: 140349099290.8913 - val_mean_absolute_error: 241179.7274
Epoch 3/500
17290/17290 [==============================] - 1s 50us/step - loss: 123384886121.2132 - mean_absolute_error: 223986.3051 - val_loss: 134083273613.8010 - val_mean_absolute_error: 224397.1317
Epoch 4/500
17290/17290 [==============================] - 1s 43us/step - loss: 115877772662.1835 - mean_absolute_error: 216129.7675 - val_loss: 122512462455.1745 - val_mean_absolute_error: 218696.0417
Epoch 5/500
17290/17290 [==============================] - 1s 45us/step - loss: 103170042606.2621 - mean_absolute_error: 202284.0079 - val_loss

Epoch 82/500
17290/17290 [==============================] - 1s 41us/step - loss: 53085972007.3846 - mean_absolute_error: 152476.5356 - val_loss: 57109776968.7367 - val_mean_absolute_error: 154120.6201
Epoch 83/500
17290/17290 [==============================] - 1s 50us/step - loss: 53382619892.1846 - mean_absolute_error: 152981.1104 - val_loss: 58211343356.6830 - val_mean_absolute_error: 151360.1900
Epoch 84/500
17290/17290 [==============================] - 1s 43us/step - loss: 53346563703.6937 - mean_absolute_error: 153003.3270 - val_loss: 57206209517.5197 - val_mean_absolute_error: 151161.1525
Epoch 85/500
17290/17290 [==============================] - 1s 73us/step - loss: 53108781978.0146 - mean_absolute_error: 152849.6674 - val_loss: 57587463889.6807 - val_mean_absolute_error: 150682.0388
Epoch 86/500
17290/17290 [==============================] - 1s 82us/step - loss: 52805414677.9429 - mean_absolute_error: 151851.2416 - val_loss: 57529827240.8107 - val_mean_absolute_error: 155549.

Epoch 123/500
17290/17290 [==============================] - 1s 52us/step - loss: 51725823819.0084 - mean_absolute_error: 150701.3399 - val_loss: 55589707677.4382 - val_mean_absolute_error: 149348.1310
Epoch 124/500
17290/17290 [==============================] - 1s 52us/step - loss: 51866800205.7032 - mean_absolute_error: 151067.2017 - val_loss: 55604773059.2281 - val_mean_absolute_error: 151809.4617
Epoch 125/500
17290/17290 [==============================] - 3s 150us/step - loss: 51548971007.0524 - mean_absolute_error: 150078.3122 - val_loss: 56408026715.2170 - val_mean_absolute_error: 154653.0422
Epoch 126/500
17290/17290 [==============================] - 3s 186us/step - loss: 51387809828.0088 - mean_absolute_error: 150211.6192 - val_loss: 58988978536.1296 - val_mean_absolute_error: 165440.1562
Epoch 127/500
17290/17290 [==============================] - 3s 159us/step - loss: 52868533712.4423 - mean_absolute_error: 152713.5058 - val_loss: 56676642127.9630 - val_mean_absolute_error:

17290/17290 [==============================] - 1s 54us/step - loss: 48602185760.6922 - mean_absolute_error: 146010.1016 - val_loss: 51671042968.2258 - val_mean_absolute_error: 148085.4525
Epoch 204/500
17290/17290 [==============================] - 1s 60us/step - loss: 48542392052.7769 - mean_absolute_error: 145742.4108 - val_loss: 51439478819.0652 - val_mean_absolute_error: 144743.3790
Epoch 205/500
17290/17290 [==============================] - 1s 71us/step - loss: 48291398220.8148 - mean_absolute_error: 145660.7170 - val_loss: 51722087869.8973 - val_mean_absolute_error: 142478.9336
Epoch 206/500
17290/17290 [==============================] - 1s 68us/step - loss: 48129054015.4596 - mean_absolute_error: 145338.1523 - val_loss: 52375774694.6488 - val_mean_absolute_error: 142385.9516
Epoch 207/500
17290/17290 [==============================] - 1s 47us/step - loss: 48704361667.0871 - mean_absolute_error: 146308.5848 - val_loss: 53325840471.1893 - val_mean_absolute_error: 148943.6884
Epoc

17290/17290 [==============================] - 1s 57us/step - loss: 46037897141.6134 - mean_absolute_error: 142700.3077 - val_loss: 48424031786.8839 - val_mean_absolute_error: 139193.6173
Epoch 284/500
17290/17290 [==============================] - 1s 64us/step - loss: 45983898849.1734 - mean_absolute_error: 143073.4674 - val_loss: 47895810116.2351 - val_mean_absolute_error: 143453.3874
Epoch 285/500
17290/17290 [==============================] - 1s 52us/step - loss: 45720661431.8047 - mean_absolute_error: 142524.2889 - val_loss: 47290944593.9769 - val_mean_absolute_error: 140919.9480
Epoch 286/500
17290/17290 [==============================] - 1s 52us/step - loss: 45781839897.8221 - mean_absolute_error: 142382.8234 - val_loss: 47852141896.3813 - val_mean_absolute_error: 140122.5011
Epoch 287/500
17290/17290 [==============================] - 1s 52us/step - loss: 45379627790.0067 - mean_absolute_error: 142093.0553 - val_loss: 49673660964.7237 - val_mean_absolute_error: 139150.5441
Epoc

17290/17290 [==============================] - 1s 48us/step - loss: 44556903910.4740 - mean_absolute_error: 140965.9388 - val_loss: 45038677077.7677 - val_mean_absolute_error: 137443.6301
Epoch 364/500
17290/17290 [==============================] - 1s 48us/step - loss: 44249555552.8329 - mean_absolute_error: 140645.5633 - val_loss: 46261181341.9121 - val_mean_absolute_error: 136893.8705
Epoch 365/500
17290/17290 [==============================] - 1s 47us/step - loss: 44075019170.3061 - mean_absolute_error: 139979.5980 - val_loss: 45834247869.7788 - val_mean_absolute_error: 140952.8173
Epoch 366/500
17290/17290 [==============================] - 1s 51us/step - loss: 44192741075.3740 - mean_absolute_error: 140342.2374 - val_loss: 46175227458.5766 - val_mean_absolute_error: 135359.0599
Epoch 367/500
17290/17290 [==============================] - 1s 65us/step - loss: 44408389705.9128 - mean_absolute_error: 140547.8631 - val_loss: 45607169431.9889 - val_mean_absolute_error: 139744.7764
Epoc

17290/17290 [==============================] - 1s 59us/step - loss: 43863779716.5159 - mean_absolute_error: 140251.2218 - val_loss: 44747391127.6335 - val_mean_absolute_error: 134511.6012
Epoch 444/500
17290/17290 [==============================] - 2s 89us/step - loss: 42655596795.9431 - mean_absolute_error: 137815.5565 - val_loss: 46809917490.2286 - val_mean_absolute_error: 138904.6394
Epoch 445/500
17290/17290 [==============================] - 1s 69us/step - loss: 43102013555.1334 - mean_absolute_error: 139040.0316 - val_loss: 44646383962.8616 - val_mean_absolute_error: 138531.4106
Epoch 446/500
17290/17290 [==============================] - 1s 59us/step - loss: 42817839145.9313 - mean_absolute_error: 138300.8853 - val_loss: 44894630977.8658 - val_mean_absolute_error: 136149.8074
Epoch 447/500
17290/17290 [==============================] - 1s 51us/step - loss: 42716567621.6486 - mean_absolute_error: 138428.6214 - val_loss: 45189992173.1643 - val_mean_absolute_error: 135521.5207
Epoc

Using the data just out of the box gives us the MAE of $137,388.15. Now let's look at some data processing techniques and how they can help improve the performance.



### Whitening the data

Some of our features are of the order of 0.1, some of the order of 10, some of the order of 100s and some of 10000. This disparity in the order of the different numeric values can cause higher order values to dominate other values. ‘Whitening’ the data helps normalize the order of the different values.

The whitening operation takes the data in the eigenbasis and divides every dimension by the eigenvalue to normalize the scale. The geometric interpretation of this transformation is that if the input data is a multivariable Gaussian, then the whitened data will be a Gaussian with zero mean and identity covariance matrix. Assuming our data should be a Gaussian distribution, we’ll compute the z-score for each column.

An important point to make about the preprocessing is that any preprocessing statistics (e.g. the data mean) must only be computed on the training data, and then applied to the validation/test data.

In [9]:
train_mean = train[cols].mean(axis=0)
train_std = train[cols].std(axis=0)
train[cols] = (train[cols] - train_mean) / train_std
validate[cols] = (validate[cols] - train_mean) / train_std
test[cols] = (test[cols] - train_mean) / train_std

Now, rerun the model

In [10]:
epochs = 500
batch = 128

cols = list(train.columns)
cols.remove(label_column)
history = t_model.fit(
    train[cols], train[label_column],
    batch_size=batch,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    validation_data=(validate[cols],validate[label_column])
    #callbacks=keras_callbacks
)
score = t_model.evaluate(test[cols], test[label_column], verbose=0)

Train on 17290 samples, validate on 2161 samples
Epoch 1/500
17290/17290 [==============================] - 1s 54us/step - loss: 394341484747.4971 - mean_absolute_error: 525231.0109 - val_loss: 397673147500.5127 - val_mean_absolute_error: 521503.5610
Epoch 2/500
17290/17290 [==============================] - 1s 65us/step - loss: 371500339267.7534 - mean_absolute_error: 511944.0628 - val_loss: 369350498050.9616 - val_mean_absolute_error: 506017.2640
Epoch 3/500
17290/17290 [==============================] - 1s 66us/step - loss: 347601065660.1578 - mean_absolute_error: 496263.4875 - val_loss: 344226689086.5488 - val_mean_absolute_error: 489489.4786
Epoch 4/500
17290/17290 [==============================] - 1s 68us/step - loss: 327258562716.3540 - mean_absolute_error: 481452.4181 - val_loss: 323257414809.0551 - val_mean_absolute_error: 473502.4171
Epoch 5/500
17290/17290 [==============================] - 2s 94us/step - loss: 310854974214.4259 - mean_absolute_error: 467960.0854 - val_loss

Epoch 82/500
17290/17290 [==============================] - 1s 52us/step - loss: 38819231312.8421 - mean_absolute_error: 126837.9170 - val_loss: 38034896887.4706 - val_mean_absolute_error: 126341.3333
Epoch 83/500
17290/17290 [==============================] - 1s 56us/step - loss: 38630573853.4052 - mean_absolute_error: 126417.2723 - val_loss: 37842904632.1518 - val_mean_absolute_error: 125879.2613
Epoch 84/500
17290/17290 [==============================] - 1s 58us/step - loss: 38449038986.8826 - mean_absolute_error: 126050.4109 - val_loss: 37714497987.1097 - val_mean_absolute_error: 125317.0830
Epoch 85/500
17290/17290 [==============================] - 1s 80us/step - loss: 38277786812.2170 - mean_absolute_error: 125693.7503 - val_loss: 37537569007.7705 - val_mean_absolute_error: 124913.3602
Epoch 86/500
17290/17290 [==============================] - 1s 59us/step - loss: 38107185866.4902 - mean_absolute_error: 125187.1289 - val_loss: 37374142190.1120 - val_mean_absolute_error: 124545.

17290/17290 [==============================] - 4s 214us/step - loss: 32478029310.4009 - mean_absolute_error: 112374.0506 - val_loss: 31416411830.1971 - val_mean_absolute_error: 111350.6785
Epoch 163/500
17290/17290 [==============================] - 3s 168us/step - loss: 32445164559.2800 - mean_absolute_error: 112415.8816 - val_loss: 31460921201.8436 - val_mean_absolute_error: 111141.5719
Epoch 164/500
17290/17290 [==============================] - 3s 154us/step - loss: 32401948593.8230 - mean_absolute_error: 112308.0364 - val_loss: 31464958738.5988 - val_mean_absolute_error: 110961.4180
Epoch 165/500
17290/17290 [==============================] - 1s 83us/step - loss: 32404939308.7149 - mean_absolute_error: 112036.4274 - val_loss: 31393915972.7089 - val_mean_absolute_error: 110907.5668
Epoch 166/500
17290/17290 [==============================] - 2s 91us/step - loss: 32347015884.5631 - mean_absolute_error: 111883.7332 - val_loss: 31323638706.7617 - val_mean_absolute_error: 110907.5276
E

17290/17290 [==============================] - 1s 50us/step - loss: 30161652857.1743 - mean_absolute_error: 107366.9295 - val_loss: 29107509130.4840 - val_mean_absolute_error: 106295.9971
Epoch 243/500
17290/17290 [==============================] - 1s 57us/step - loss: 30136741090.8317 - mean_absolute_error: 107033.1650 - val_loss: 28975003117.2827 - val_mean_absolute_error: 106381.2685
Epoch 244/500
17290/17290 [==============================] - 1s 51us/step - loss: 30118049813.5579 - mean_absolute_error: 107219.2571 - val_loss: 29027438009.1587 - val_mean_absolute_error: 106235.8614
Epoch 245/500
17290/17290 [==============================] - 1s 61us/step - loss: 30113282855.5919 - mean_absolute_error: 107108.3467 - val_loss: 28947508715.3873 - val_mean_absolute_error: 106249.3934
Epoch 246/500
17290/17290 [==============================] - 1s 75us/step - loss: 30064608859.7395 - mean_absolute_error: 106872.9088 - val_loss: 28871750178.8283 - val_mean_absolute_error: 106332.1233
Epoc

17290/17290 [==============================] - 1s 49us/step - loss: 28811395568.0685 - mean_absolute_error: 104638.7967 - val_loss: 27685361413.8047 - val_mean_absolute_error: 103716.4598
Epoch 323/500
17290/17290 [==============================] - 1s 54us/step - loss: 28796739055.2393 - mean_absolute_error: 104433.0394 - val_loss: 27603524427.4614 - val_mean_absolute_error: 103771.3971
Epoch 324/500
17290/17290 [==============================] - 1s 60us/step - loss: 28792056239.3948 - mean_absolute_error: 104527.7273 - val_loss: 27546046968.1814 - val_mean_absolute_error: 103753.3837
Epoch 325/500
17290/17290 [==============================] - 1s 43us/step - loss: 28767548237.4959 - mean_absolute_error: 104481.3540 - val_loss: 27591522781.6455 - val_mean_absolute_error: 103616.6602
Epoch 326/500
17290/17290 [==============================] - 1s 79us/step - loss: 28750284283.3212 - mean_absolute_error: 104488.3574 - val_loss: 27688443400.7663 - val_mean_absolute_error: 103529.4773
Epoc

17290/17290 [==============================] - 1s 48us/step - loss: 27753319361.6361 - mean_absolute_error: 102518.4276 - val_loss: 26690281655.8556 - val_mean_absolute_error: 101754.9879
Epoch 403/500
17290/17290 [==============================] - 1s 79us/step - loss: 27709853340.6501 - mean_absolute_error: 102257.4045 - val_loss: 26577628559.4595 - val_mean_absolute_error: 101839.2666
Epoch 404/500
17290/17290 [==============================] - 1s 49us/step - loss: 27710169020.1282 - mean_absolute_error: 102364.9453 - val_loss: 26648003778.7543 - val_mean_absolute_error: 101729.1961
Epoch 405/500
17290/17290 [==============================] - 1s 69us/step - loss: 27696388272.7274 - mean_absolute_error: 102321.6454 - val_loss: 26655866296.2110 - val_mean_absolute_error: 101669.6936
Epoch 406/500
17290/17290 [==============================] - 1s 43us/step - loss: 27684508954.2663 - mean_absolute_error: 102169.3063 - val_loss: 26551970725.4938 - val_mean_absolute_error: 101732.8675
Epoc

Epoch 483/500
17290/17290 [==============================] - 1s 56us/step - loss: 26708888257.8434 - mean_absolute_error: 100093.4303 - val_loss: 25793458471.6853 - val_mean_absolute_error: 99756.6437
Epoch 484/500
17290/17290 [==============================] - 1s 57us/step - loss: 26677111537.3418 - mean_absolute_error: 100053.1116 - val_loss: 25871932427.8464 - val_mean_absolute_error: 99684.6897
Epoch 485/500
17290/17290 [==============================] - 1s 56us/step - loss: 26668889829.0230 - mean_absolute_error: 100021.8992 - val_loss: 25791763387.2911 - val_mean_absolute_error: 99700.4618
Epoch 486/500
17290/17290 [==============================] - 1s 64us/step - loss: 26659250969.6148 - mean_absolute_error: 100026.8683 - val_loss: 25808864580.5905 - val_mean_absolute_error: 99660.7644
Epoch 487/500
17290/17290 [==============================] - 2s 92us/step - loss: 26651491877.0156 - mean_absolute_error: 99905.8506 - val_loss: 25676600408.6108 - val_mean_absolute_error: 99657.3

Once we re-run the model with the normalized data, our MAE does down to 99,119.00. This is a huge improvement over using the data as is.

### Converting ‘long’ and ‘lat’ to distance

While longitude and latitude can be used as is, we can get some more insight by transforming longitude and latitude into the distance of the house from a fixed location and booleans representing the direction where the house sits relative to the fixed location.

In [11]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r
  
FIXED_LONG = -122.213896
FIXED_LAT = 47.560053
kc_raw_data['distance'] = kc_raw_data.apply(lambda row: haversine(FIXED_LONG, FIXED_LAT, row['long'], row['lat']), axis=1)
kc_raw_data['greater_long'] = (kc_raw_data['long'] >= FIXED_LONG).astype(int)
kc_raw_data['less_long'] = (kc_raw_data['long'] < FIXED_LONG).astype(int)
kc_raw_data['greater_lat'] = (kc_raw_data['lat'] >= FIXED_LAT).astype(int)
kc_raw_data['less_lat'] = (kc_raw_data['lat'] < FIXED_LAT).astype(int)

In [12]:
kc_raw_data.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sale_yr', 'sale_month',
       'sale_day', 'distance', 'greater_long', 'less_long', 'greater_lat',
       'less_lat'],
      dtype='object')

In [13]:
kc_data = pd.DataFrame(kc_raw_data,columns=[
        'sale_yr','sale_month','sale_day', 'view', 'waterfront', 'distance', 'greater_long',    'greater_lat','less_long','less_lat','bedrooms','bathrooms','sqft_living','sqft_lot','floors',
        'condition','grade','sqft_above','sqft_basement','yr_built','yr_renovated',
        'zipcode','sqft_living15','sqft_lot15','price'])

In [14]:
kc_data = kc_data.sample(frac=1)
train = kc_data.sample(frac=0.8)
print('Train:' + str(train.shape))
validate = kc_data.sample(frac=0.1)
print('Validate:' + str(validate.shape))
test = kc_data.sample(frac=0.1)
print('Test:' + str(test.shape))
xsize = kc_data.shape[1]-1
ysize = 1
print ('xsize:' + str(xsize))

Train:(17290, 25)
Validate:(2161, 25)
Test:(2161, 25)
xsize:24


In [15]:
cols = list(train.columns)
cols.remove(label_column)
train_mean = train[cols].mean(axis=0)
train_std = train[cols].std(axis=0)
train[cols] = (train[cols] - train_mean) / train_std
validate[cols] = (validate[cols] - train_mean) / train_std
test[cols] = (test[cols] - train_mean) / train_std

In [16]:
t_model2 = Sequential()
t_model2.add(Dense(100, activation="relu",input_shape=(xsize,)))
t_model2.add(Dense(50,activation="relu"))
t_model2.add(Dense(1,kernel_initializer='normal'))
t_model2.compile(
    loss="mean_squared_error",
    optimizer=Adam(lr=0.001),
    metrics=[metrics.mae]
)

In [17]:
t_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               2500      
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 51        
Total params: 7,601
Trainable params: 7,601
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = t_model2.fit(
    train[cols], train[label_column],
    batch_size=batch,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    validation_data=(validate[cols],validate[label_column])
    #callbacks=keras_callbacks
)
score = t_model2.evaluate(test[cols], test[label_column], verbose=0)

Train on 17290 samples, validate on 2161 samples
Epoch 1/500
17290/17290 [==============================] - 1s 78us/step - loss: 427297470438.6517 - mean_absolute_error: 540360.2606 - val_loss: 418281612814.4526 - val_mean_absolute_error: 541054.8782
Epoch 2/500
17290/17290 [==============================] - 1s 54us/step - loss: 426178095890.2709 - mean_absolute_error: 539687.7405 - val_loss: 415696374873.5585 - val_mean_absolute_error: 539503.4588
Epoch 3/500
17290/17290 [==============================] - 2s 101us/step - loss: 421017678267.9505 - mean_absolute_error: 536658.4734 - val_loss: 407515609748.5535 - val_mean_absolute_error: 534690.3645
Epoch 4/500
17290/17290 [==============================] - 1s 63us/step - loss: 408822162983.8585 - mean_absolute_error: 529549.8946 - val_loss: 391078376487.3300 - val_mean_absolute_error: 524954.4922
Epoch 5/500
17290/17290 [==============================] - 1s 51us/step - loss: 387350141110.1761 - mean_absolute_error: 516830.2185 - val_los

Epoch 82/500
17290/17290 [==============================] - 2s 95us/step - loss: 25298894214.0558 - mean_absolute_error: 100537.0609 - val_loss: 21020646462.5488 - val_mean_absolute_error: 98006.9517
Epoch 83/500
17290/17290 [==============================] - 2s 91us/step - loss: 25221327439.7761 - mean_absolute_error: 100439.1895 - val_loss: 20986307712.8885 - val_mean_absolute_error: 97866.7050
Epoch 84/500
17290/17290 [==============================] - 1s 59us/step - loss: 25168256588.5779 - mean_absolute_error: 100423.5098 - val_loss: 20937807247.4595 - val_mean_absolute_error: 97618.6259
Epoch 85/500
17290/17290 [==============================] - 1s 71us/step - loss: 25112879238.6776 - mean_absolute_error: 100294.2770 - val_loss: 20898357007.2818 - val_mean_absolute_error: 97424.8307
Epoch 86/500
17290/17290 [==============================] - 1s 50us/step - loss: 25065710129.9267 - mean_absolute_error: 100119.3589 - val_loss: 20879108427.6983 - val_mean_absolute_error: 97472.8020


17290/17290 [==============================] - 2s 102us/step - loss: 23690340821.6541 - mean_absolute_error: 96929.3732 - val_loss: 19947925268.0204 - val_mean_absolute_error: 94034.9486
Epoch 124/500
17290/17290 [==============================] - 3s 161us/step - loss: 23666874317.0665 - mean_absolute_error: 96623.3019 - val_loss: 19936170146.0583 - val_mean_absolute_error: 94060.1850
Epoch 125/500
17290/17290 [==============================] - 1s 74us/step - loss: 23632462511.2467 - mean_absolute_error: 96638.7997 - val_loss: 19915241196.2166 - val_mean_absolute_error: 93953.1221
Epoch 126/500
17290/17290 [==============================] - 2s 100us/step - loss: 23621667568.6311 - mean_absolute_error: 96692.1578 - val_loss: 19880435999.1559 - val_mean_absolute_error: 93726.6463
Epoch 127/500
17290/17290 [==============================] - 1s 82us/step - loss: 23581116948.3142 - mean_absolute_error: 96410.3577 - val_loss: 19858498852.8422 - val_mean_absolute_error: 93658.0323
Epoch 128/5

17290/17290 [==============================] - 1s 64us/step - loss: 22696164764.7389 - mean_absolute_error: 94329.1244 - val_loss: 19299564066.8283 - val_mean_absolute_error: 91449.2431
Epoch 165/500
17290/17290 [==============================] - 1s 62us/step - loss: 22686848980.1735 - mean_absolute_error: 94261.7139 - val_loss: 19289242600.3073 - val_mean_absolute_error: 91434.6214
Epoch 166/500
17290/17290 [==============================] - 2s 88us/step - loss: 22656006429.2275 - mean_absolute_error: 94072.2552 - val_loss: 19288060853.6048 - val_mean_absolute_error: 91498.8113
Epoch 167/500
17290/17290 [==============================] - 2s 114us/step - loss: 22638916866.1025 - mean_absolute_error: 94144.2656 - val_loss: 19272580806.7820 - val_mean_absolute_error: 91425.1021
Epoch 168/500
17290/17290 [==============================] - 2s 99us/step - loss: 22614506498.7243 - mean_absolute_error: 93924.6556 - val_loss: 19274355866.0028 - val_mean_absolute_error: 91492.1962
Epoch 169/500

17290/17290 [==============================] - 1s 68us/step - loss: 22002473701.4968 - mean_absolute_error: 92654.6210 - val_loss: 18891899646.6969 - val_mean_absolute_error: 90035.8980
Epoch 206/500
17290/17290 [==============================] - 1s 60us/step - loss: 21981483538.0636 - mean_absolute_error: 92605.6580 - val_loss: 18904368895.6446 - val_mean_absolute_error: 90166.2238
Epoch 207/500
17290/17290 [==============================] - 1s 55us/step - loss: 21976111775.1375 - mean_absolute_error: 92654.2983 - val_loss: 18884441360.4664 - val_mean_absolute_error: 90031.8380
Epoch 208/500
17290/17290 [==============================] - 1s 48us/step - loss: 21964569601.5398 - mean_absolute_error: 92549.4280 - val_loss: 18879704938.2619 - val_mean_absolute_error: 90038.3454
Epoch 209/500
17290/17290 [==============================] - 1s 77us/step - loss: 21949002130.1377 - mean_absolute_error: 92501.8752 - val_loss: 18856340877.0902 - val_mean_absolute_error: 89909.2460
Epoch 210/500


...

17290/17290 [==============================] - 1s 43us/step - loss: 20810962701.6514 - mean_absolute_error: 90502.6533 - val_loss: 18295507878.9153 - val_mean_absolute_error: 88099.1108
Epoch 329/500
17290/17290 [==============================] - 1s 86us/step - loss: 20794066017.0105 - mean_absolute_error: 90576.3632 - val_loss: 18278751886.8672 - val_mean_absolute_error: 87939.3849
Epoch 330/500
17290/17290 [==============================] - 1s 69us/step - loss: 20791290982.0447 - mean_absolute_error: 90393.8730 - val_loss: 18283297650.3174 - val_mean_absolute_error: 88071.7127
Epoch 331/500
17290/17290 [==============================] - 1s 70us/step - loss: 20783937583.9722 - mean_absolute_error: 90470.0569 - val_loss: 18276931473.5919 - val_mean_absolute_error: 88036.4344
Epoch 332/500
17290/17290 [==============================] - 1s 58us/step - loss: 20782389217.9729 - mean_absolute_error: 90408.2007 - val_loss: 18287419742.1786 - val_mean_absolute_error: 88118.0639
Epoch 333/500


Epoch 369/500
17290/17290 [==============================] - 1s 57us/step - loss: 20534205628.3355 - mean_absolute_error: 90097.1290 - val_loss: 18156732191.8667 - val_mean_absolute_error: 87689.1292
Epoch 370/500
17290/17290 [==============================] - 1s 48us/step - loss: 20530091238.6221 - mean_absolute_error: 90048.7687 - val_loss: 18157193529.2180 - val_mean_absolute_error: 87656.9642
Epoch 371/500
17290/17290 [==============================] - 1s 82us/step - loss: 20535496722.6559 - mean_absolute_error: 89997.5862 - val_loss: 18166148266.1138 - val_mean_absolute_error: 87747.7215
Epoch 372/500
17290/17290 [==============================] - 1s 59us/step - loss: 20524846112.2776 - mean_absolute_error: 90062.8614 - val_loss: 18159494289.9472 - val_mean_absolute_error: 87718.3812
Epoch 373/500
17290/17290 [==============================] - 1s 55us/step - loss: 20512728340.1069 - mean_absolute_error: 89945.1534 - val_loss: 18154800443.5872 - val_mean_absolute_error: 87760.9533


Epoch 410/500
17290/17290 [==============================] - 1s 72us/step - loss: 20306945110.3500 - mean_absolute_error: 89599.4430 - val_loss: 18030258838.4489 - val_mean_absolute_error: 87329.7262
Epoch 411/500
17290/17290 [==============================] - 1s 53us/step - loss: 20299595058.6670 - mean_absolute_error: 89614.3789 - val_loss: 18025126925.7418 - val_mean_absolute_error: 87315.2095
Epoch 412/500
17290/17290 [==============================] - 1s 54us/step - loss: 20289840745.7166 - mean_absolute_error: 89731.2058 - val_loss: 18032305167.1633 - val_mean_absolute_error: 87342.4730
Epoch 413/500
17290/17290 [==============================] - 1s 55us/step - loss: 20293581902.7692 - mean_absolute_error: 89601.5010 - val_loss: 18036266494.3415 - val_mean_absolute_error: 87336.2558
Epoch 414/500
17290/17290 [==============================] - 1s 52us/step - loss: 20281348062.5971 - mean_absolute_error: 89676.7892 - val_loss: 18032389229.9343 - val_mean_absolute_error: 87344.5558


Epoch 451/500
17290/17290 [==============================] - 1s 48us/step - loss: 20090663948.4372 - mean_absolute_error: 89258.9398 - val_loss: 17936971805.3790 - val_mean_absolute_error: 87005.3281
Epoch 452/500
17290/17290 [==============================] - 1s 61us/step - loss: 20092246270.0752 - mean_absolute_error: 89325.2190 - val_loss: 17925337066.2027 - val_mean_absolute_error: 86916.4669
Epoch 453/500
17290/17290 [==============================] - 1s 84us/step - loss: 20098612570.2293 - mean_absolute_error: 89323.6475 - val_loss: 17928889174.8339 - val_mean_absolute_error: 86899.1545
Epoch 454/500
17290/17290 [==============================] - 1s 46us/step - loss: 20081138173.0980 - mean_absolute_error: 89158.4891 - val_loss: 17920875867.8093 - val_mean_absolute_error: 86922.5891
Epoch 455/500
17290/17290 [==============================] - 1s 49us/step - loss: 20078307811.5128 - mean_absolute_error: 89225.7711 - val_loss: 17930796271.7705 - val_mean_absolute_error: 86934.8588


Epoch 492/500
17290/17290 [==============================] - 1s 66us/step - loss: 19908060045.1479 - mean_absolute_error: 89005.6785 - val_loss: 17813440588.7645 - val_mean_absolute_error: 86587.0432
Epoch 493/500
17290/17290 [==============================] - 1s 67us/step - loss: 19897070785.6657 - mean_absolute_error: 88933.1908 - val_loss: 17819968983.4854 - val_mean_absolute_error: 86598.2835
Epoch 494/500
17290/17290 [==============================] - 1s 59us/step - loss: 19896044835.9792 - mean_absolute_error: 88973.6559 - val_loss: 17828044552.1740 - val_mean_absolute_error: 86623.9032
Epoch 495/500
17290/17290 [==============================] - 1s 59us/step - loss: 19900272981.3136 - mean_absolute_error: 89011.1045 - val_loss: 17810388921.8695 - val_mean_absolute_error: 86452.6784
Epoch 496/500
17290/17290 [==============================] - 1s 48us/step - loss: 19895274586.9696 - mean_absolute_error: 88914.3175 - val_loss: 17812150951.0338 - val_mean_absolute_error: 86513.1486


Using the new features ‘distance’, ‘greater_long’, ‘less_long’, ‘greater_lat’ and ‘less_lat’, we train the model again over the dataset. The resulting MAE over the training set goes further down to $86,617.94.

### For code associated with categorical embeddings, see notebook regressioncategoricalembeddings